### Extract Data from PDF Files with Python

In [22]:
# !pip install PDFPlumber
import pdfplumber
import pandas as pd
import os

In [13]:
import pyodbc

server = 'ASUSTUF\SQL22'
database = 'AdventureWorksDW2019'
# username = 'nombre_usuario'
# password = 'contraseña'

try:
    connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                                f'SERVER={server};'
                                f'DATABASE={database};'
                                f'Trusted_Connection=yes;')
    print('Conexión exitosa')
except pyodbc.Error as e:
    print(f'Error al conectar a la base de datos: {str(e)}')

Conexión exitosa


In [15]:
query = "SELECT * FROM vFactFinance"
df_query = pd.read_sql(query, connection)
df_query.head()

C:\Users\rfigu\AppData\Local\Temp\ipykernel_4440\812222591.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_query = pd.read_sql(query, connection)


,FinanceKey,Fecha,Organizacion,Gerencia,Escenario,Descripcion Cuenta,Monto,TotalYTD
0,26702,2012-12-28,Australia,Corporate,Actual,Salaries,19305.12,1392505.88
1,26703,2012-12-28,Australia,Corporate,Actual,Payroll Taxes,1783.74,1392505.88
2,26704,2012-12-28,Australia,Corporate,Actual,Employee Benefits,1188.99,1392505.88
3,26705,2012-12-28,Australia,Corporate,Actual,Travel Transportation,190.73,1392505.88
4,26706,2012-12-28,Australia,Corporate,Actual,Travel Lodging,173.18,1392505.88


Extraer datos de la tabla de la factura PDF

In [ ]:
root_dir = os.getcwd()

print("Ruta base archivos: {}".format(root_dir)) 

df_archivos = []
ciclo = 0

for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.pdf'):
            file_paths = os.path.join(dirpath, filename)
            # Se utiliza pdfPlumber para leer los datos de la tabla factura
            with pdfplumber.open(file_paths) as temp:
                print(file_paths)
                ciclo += 1
                    # Nombre de pdf dinamico
                print(f"pdf_{ciclo}")
                first_page = temp.pages[0]
                # print(first_page.extract_table())

In [3]:
diccColumnas = ('Codigo', 'Descripcion', 'Unidad de nmedida', 'Cantidad', 'Precio Unitario', 'Descuento', 'Exentas', 'Monto 5%', 'Monto 10%', 'NroFactura', 'FechaEmision')
diccColumnas

('Codigo',
 'Descripcion',
 'Unidad de nmedida',
 'Cantidad',
 'Precio Unitario',
 'Descuento',
 'Exentas',
 'Monto 5%',
 'Monto 10%',
 'NroFactura',
 'FechaEmision')

In [4]:
root_dir = os.getcwd()

df_archivos = []
ciclo = 0

for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.pdf'):
            file_paths = os.path.join(dirpath, filename)
            # Se utiliza pdfPlumber para leer los datos de la tabla factura
            with pdfplumber.open(file_paths) as temp:
                print(file_paths)
                ciclo += 1
                    # Nombre de pdf dinamico
                print(f"pdf_{ciclo}")
                first_page = temp.pages[0]
                        # print(first_page.extract_table())
                dfnames = first_page.extract_table()
                df_t = pd.DataFrame(dfnames)
                headers = df_t.iloc[0].values
                df_t.columns = headers
                df_t.drop(index=0, axis=0, inplace=True)
                df_t = df_t.dropna()

                    # Se agrega el Nro de Factura al Df
                texto = first_page.extract_text()
                nroFact  = texto[0:15]
                print("Nro Factura: {}".format(nroFact)) 
                df_t['NroFactura']=nroFact

                # #Se agrega la fecha y Hora impresa en la factura
                lines = texto.split('\n')
                filtered_lines = [line for line in lines if 'Fecha y hora' in line]
                # print('\n'.join(filtered_lines))
                str(filtered_lines)
                fechafact =str(filtered_lines)[27:37]
                print("Fecha Emision: {}".format(fechafact))
                df_t['FechaEmision']=fechafact
                df_archivos.append(df_t)
                
            df = pd.concat(df_archivos, ignore_index=True)
            df.columns = diccColumnas


c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\29_Octubre_2023.pdf
pdf_1
Nro Factura: 001-001-0036986
Fecha Emision: 10-11-2022
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\30_Diciembre_2022.pdf
pdf_2
Nro Factura: 001-001-0069238
Fecha Emision: 30-01-2023
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\31_DicExtraordinario_2022.pdf
pdf_3
Nro Factura: 001-001-0069237
Fecha Emision: 30-01-2023
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\32_Enero_2023.pdf
pdf_4
Nro Factura: 001-001-0092791
Fecha Emision: 01-04-2023
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\33_Febrero_2023.pdf
pdf_5
Nro Factura: 001-001-0098732
Fecha Emision: 19-04-2023
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\34_Mayo_2023.pdf
pdf_6
Nro Factura: 001-001-0106414
Fecha Emision: 10-05-2023
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\35_Junio_2023.pdf
pdf_7
Nro Factura: 001-001-0118558
Fecha Emision: 13-06-2023
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\36_Julio_2023.pdf
pdf_8
Nro Factura: 001-001-0132813

In [16]:
df.head()

,Codigo,Descripcion,Unidad de nmedida,Cantidad,Precio Unitario,Descuento,Exentas,Monto 5%,Monto 10%,NroFactura,FechaEmision
0,ESPERA20EX,Espera - ESPERA_EXENTA Oct/22 22:240 - Cto.: 1...,UNI,1.00,1.088.000.0,0.00,1.088.000.0,,,001-001-0036986,10-11-2022
1,ESPERA20IM,Gastos Administrativos,UNI,1.00,272.000.0,0.00,,,272.000.0,001-001-0036986,10-11-2022
2,ESPERA20EX,Espera - ESPERA_EXENTA Nov/22 23:240 - Cto.:\n...,UNI,1.00,1.088.000.0,0.00,1.088.000.0,,,001-001-0069238,30-01-2023
3,ESPERA20IM,Gastos Administrativos,UNI,1.00,272.000.0,0.00,,,272.000.0,001-001-0069238,30-01-2023
4,CUOTAEXTIM,EXTRA.ESPERA_GRAVADA Dic/22 5:20 - Cto.: 13467,UNI,1.00,680.000.0,0.00,,,680.000.0,001-001-0069237,30-01-2023


In [14]:
import sqlalchemy as sa

# Replace these with your actual server name and database name
server_name = 'ASUSTUF\SQL22'
database_name = 'AdventureWorksDW2019'

# Create the engine
engine = sa.create_engine('mssql+pyodbc://' + server_name + '/' + database_name + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')

In [17]:
# Execute a SQL query
output = engine.query("SELECT * FROM vFactFinance")

type(output)

AttributeError: 'Engine' object has no attribute 'query'

In [4]:


# Use pandas to execute the query and store the result in a DataFrame
df_W = pd.read_sql(query, engine)

# Now df is a DataFrame containing the result of the query
df_W

AttributeError: 'OptionEngine' object has no attribute 'execute'

In [17]:
# Escribir el DataFrame en la base de datos SQL
df.to_sql('mi_tabla_temp', connection, if_exists='replace')

C:\Users\rfigu\AppData\Local\Temp\ipykernel_4440\76403246.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('mi_tabla_temp', connection, if_exists='replace')


DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]El nombre de objeto 'sqlite_master' no es válido. (208) (SQLExecDirectW); [42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]No se puede preparar la instrucción o instrucciones. (8180)")

In [ ]:
# Comparar el DataFrame cargado con el DataFrame original
assert df_loaded.equals(df), "Los DataFrames no son iguales"

### Bloque para leer la factura electronica version 1
Comprende de Marzo de 2022 a Agosto 2022

In [ ]:
rutaEspec

In [6]:
rutaEspec = r"C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1"

df_archivos_v1 = []
ciclo = 0

for dirpath, dirnames, filenames in os.walk(rutaEspec):
    for filename in filenames:
        if filename.endswith('.pdf'):
            file_paths = os.path.join(dirpath, filename)
            # Se utiliza pdfPlumber para leer los datos de la tabla factura
            with pdfplumber.open(file_paths) as temp:
                print(file_paths)
                ciclo += 1
                    # Nombre de pdf dinamico
                print(f"pdf_{ciclo}")
                first_page = temp.pages[0]
                        # print(first_page.extract_table())
                dfnames = first_page.extract_table()
                df_t = pd.DataFrame(dfnames)
                headers = df_t.iloc[0].values
                df_t.columns = headers
                df_t.drop(index=0, axis=0, inplace=True)
                df_t = df_t.dropna()

                    # Se agrega el Nro de Factura al Df # factura version 1
                texto = first_page.extract_text()
                nroFact  = texto[327:342]
                print("Nro Factura: {}".format(nroFact)) 
                df_t['NroFactura']=nroFact

                #   #Se agrega la fecha y Hora impresa en la factura version 1
                lines = texto.split('\n')
                filtered_lines = [line for line in lines if 'Fecha de Emisión:' in line]
                fechafact =str(filtered_lines)[20:30]
                print("Fecha Emision: {}".format(fechafact))
                df_t['FechaEmision']=fechafact
                df_archivos_v1.append(df_t)
                            # if ciclo == 2:
                            #     break    # break here
                
            df_v1 = pd.concat(df_archivos_v1, ignore_index=True)
            df_v1.columns = diccColumnas

C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1\22_Marzo2022.pdf
pdf_1
Nro Factura: 001-002-0010793
Fecha Emision: 10/03/2022
C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1\23_Abril2022.pdf
pdf_2
Nro Factura: 001-001-0010087
Fecha Emision: 11/04/2022
C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1\24_Mayo_2022.pdf
pdf_3
Nro Factura: 001-004-0008842
Fecha Emision: 11/05/2022
C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1\25_Junio_2022.pdf
pdf_4
Nro Factura: 001-003-0003504
Fecha Emision: 10/06/2022
C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1\26_Julio_2022.pdf
pdf_5
Nro Factura: 001-003-0005000
Fecha Emision: 12/07/2022
C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1\26_Junio_2022_Extra.pdf
pdf_6
Nro Factura: 001-001-0014565
Fecha Emision: 13/06/2022
C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1\27_Agosto_2022.pdf
pdf_7
Nro Factura: 001-001-0018137
Fecha Emision: 11/08/2022


In [7]:
df_v1

,Codigo,Descripcion,Unidad de nmedida,Cantidad,Precio Unitario,Descuento,Exentas,Monto 5%,Monto 10%,NroFactura,FechaEmision
0,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 03/22 : 15 - Cto.: 1...,Unidad\nUnidad,1.000000\n1.000000,247.000\n988.000,0\n0,0\n988.000,0\n0,247.000\n0,001-002-0010793,10/03/2022
1,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 04/22 : 16 - Cto.: 1...,Unidad\nUnidad,1.000000\n1.000000,247.000\n988.000,0\n0,0\n988.000,0\n0,247.000\n0,001-001-0010087,11/04/2022
2,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 05/22 : 17 - Cto.: 1...,Unidad\nUnidad,1.0000\n1.0000,258.00\n1.032.00,0 0\n0 0,0\n1.032.000,0\n0,258.000\n0,001-004-0008842,11/05/2022
3,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 06/22 : 18 - Cto.: 1...,Unidad\nUnidad,1.0000\n1.0000,258.00\n1.032.00,0 0\n0 0,0\n1.032.000,0\n0,258.000\n0,001-003-0003504,10/06/2022
4,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 07/22 : 19 - Cto.: 1...,Unidad\nUnidad,1.0000\n1.0000,258.00\n1.032.00,0 0\n0 0,0\n1.032.000,0\n0,258.000\n0,001-003-0005000,12/07/2022
5,CUOTAEXTIM Cu,ota Ext. 06/22 : 4 - Cto.: 13467,Unidad,1.0000,645.00,0 0,0,0,645.000,001-001-0014565,13/06/2022
6,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 08/22 : 20 - Cto.: 1...,Unidad\nUnidad,1.0000\n1.0000,258.00\n1.032.00,0 0\n0 0,0\n1.032.000,0\n0,258.000\n0,001-001-0018137,11/08/2022


In [11]:
df_v1.replace({"0\n": " ", "\n": " "}, inplace=True)

In [12]:
df_v1

,Codigo,Descripcion,Unidad de nmedida,Cantidad,Precio Unitario,Descuento,Exentas,Monto 5%,Monto 10%,NroFactura,FechaEmision
0,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 03/22 : 15 - Cto.: 1...,Unidad\nUnidad,1.000000\n1.000000,247.000\n988.000,0\n0,0\n988.000,0\n0,247.000\n0,001-002-0010793,10/03/2022
1,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 04/22 : 16 - Cto.: 1...,Unidad\nUnidad,1.000000\n1.000000,247.000\n988.000,0\n0,0\n988.000,0\n0,247.000\n0,001-001-0010087,11/04/2022
2,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 05/22 : 17 - Cto.: 1...,Unidad\nUnidad,1.0000\n1.0000,258.00\n1.032.00,0 0\n0 0,0\n1.032.000,0\n0,258.000\n0,001-004-0008842,11/05/2022
3,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 06/22 : 18 - Cto.: 1...,Unidad\nUnidad,1.0000\n1.0000,258.00\n1.032.00,0 0\n0 0,0\n1.032.000,0\n0,258.000\n0,001-003-0003504,10/06/2022
4,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 07/22 : 19 - Cto.: 1...,Unidad\nUnidad,1.0000\n1.0000,258.00\n1.032.00,0 0\n0 0,0\n1.032.000,0\n0,258.000\n0,001-003-0005000,12/07/2022
5,CUOTAEXTIM Cu,ota Ext. 06/22 : 4 - Cto.: 13467,Unidad,1.0000,645.00,0 0,0,0,645.000,001-001-0014565,13/06/2022
6,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 08/22 : 20 - Cto.: 1...,Unidad\nUnidad,1.0000\n1.0000,258.00\n1.032.00,0 0\n0 0,0\n1.032.000,0\n0,258.000\n0,001-001-0018137,11/08/2022


In [ ]:
# Nro de factura en la factra v2
texto = first_page.extract_text()
nroFact  = texto[327:342]
print("Nro Factura: {}".format(nroFact)) 

In [ ]:
lines = texto.split('\n')
filtered_lines = [line for line in lines if 'Fecha de Emisión:' in line]
fechafact =str(filtered_lines)[20:30]
print("Fecha Emision: {}".format(fechafact))